Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

### 0. Clone the repo

In [9]:
#Clone the tortoise-tts repo
!git clone https://github.com/jnordberg/tortoise-tts.git

fatal: destination path 'tortoise-tts' already exists and is not an empty directory.


### 1. Install required dependencies 

In [10]:
# the scipy version packaged with colab is not tolerant of misformated WAV files.
# install the latest version.
!pip3 install -U scipy
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/content/tortoise-tts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numba-0.48.0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached ffmpeg-1.4-py3-none-any.whl
  Using cached llvmlite-0.31.0.tar.gz (110 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to determine which version is compatible with other requirements. This could take a while.
  Using cached librosa-0.10.0.post1-py3-none-any.whl (252 kB)
  Using cached librosa-0.10.0-py3-none-any.whl (252 kB)
  Using cached librosa-0.9.2-py3-none-any.whl (214 kB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
INFO: pip is looking at multiple versions of resampy to determine which version is compatible with other requirements. This could take a while.
  Using cached resampy-0.4.1-py3-none-any.whl (3

### 2. Import the installed package

In [11]:
# Imports used through the rest of the notebook.
import os
import torch
import IPython
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
from google.colab import files
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

### 3. Load the Tortoise Model

In [12]:
# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

### 4. Upload the sample WAV files

In [13]:
# Optionally, upload use your own voice by running the next two cells. I recommend
# you upload at least 2 audio clips (if the clips increase the accuracy as well). They must be a WAV file, 6-10 seconds long.
# Define the custom voice name
CUSTOM_VOICE_NAME = "aman"

# Create the custom voice folder
custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder, exist_ok=True)

# Upload the audio files
for i, file_data in enumerate(files.upload().values()):
    with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
        f.write(file_data)

Saving Sample_1.wav to Sample_1 (1).wav
Saving Sample_2.wav to Sample_2 (1).wav
Saving Sample_3.wav to Sample_3 (1).wav
Saving Sample_4.wav to Sample_4 (1).wav
Saving Sample_5.wav to Sample_5 (1).wav


### 5. Input the text

In [14]:
# This is the text that will be spoken.
text = "Hello, my name is Aman. And, This is my first try output. Thank you Jordi"
# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "high_quality"

### 6. Gnerate the clone

In [15]:
# Generate speech with the custotm voice.
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [01:32<00:00,  5.81s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
100%|██████████| 16/16 [00:28<00:00,  1.80s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [01:03<00:00,  6.30it/s]


### 7. Save the model

In [ ]:
import pickle
model_path = "/content/model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(tts, f)

### 8. Use the saved 

In [ ]:
model_path = "/content/model.pkl"
with open(model_path, "rb") as f:
    tts = pickle.load(f)